In [50]:
import pandas as pd
from pathlib import Path

# Works whether you're running from repo root or from /notebooks
ROOT = Path.cwd()
if not (ROOT / "data" / "raw").exists() and (ROOT.parent / "data" / "raw").exists():
    ROOT = ROOT.parent

RAW = ROOT / "data" / "raw"

# Load from the actual sheet names in your file
data = pd.read_excel(RAW / "ethiopia_fi_unified_data.xlsx", sheet_name="ethiopia_fi_unified_data")
impact_links = pd.read_excel(RAW / "ethiopia_fi_unified_data.xlsx", sheet_name="Impact_sheet")

# Reference codes
reference_codes = pd.read_excel(RAW / "reference_codes.xlsx")

# Enrichment guide (all sheets)
guide = pd.read_excel(RAW / "Additional Data Points Guide.xlsx", sheet_name=None)

print("ROOT:", ROOT)
print("data:", data.shape)
print("impact_links:", impact_links.shape)
print("reference_codes:", reference_codes.shape)
print("guide sheets:", list(guide.keys()))

ROOT: c:\Users\Hp\Desktop\ethiopia-fi-forecast
data: (43, 34)
impact_links: (14, 35)
reference_codes: (71, 4)
guide sheets: ['A. Alternative Baselines', 'B. Direct Corrln', 'C. Indirect Corrln', 'D. Market Naunces']


In [51]:
data.columns

Index(['record_id', 'record_type', 'category', 'pillar', 'indicator',
       'indicator_code', 'indicator_direction', 'value_numeric', 'value_text',
       'value_type', 'unit', 'observation_date', 'period_start', 'period_end',
       'fiscal_year', 'gender', 'location', 'region', 'source_name',
       'source_type', 'source_url', 'confidence', 'related_indicator',
       'relationship_type', 'impact_direction', 'impact_magnitude',
       'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country',
       'collected_by', 'collection_date', 'original_text', 'notes'],
      dtype='str')

In [52]:
data["record_type"].value_counts(dropna=False)

record_type
observation    30
event          10
target          3
Name: count, dtype: int64

In [53]:
data.groupby(["record_type", "pillar"]).size().sort_values(ascending=False)

record_type  pillar       
observation  ACCESS           14
             USAGE            11
             GENDER            4
target       ACCESS            2
observation  AFFORDABILITY     1
target       GENDER            1
dtype: int64

In [54]:
data["source_type"].value_counts(dropna=False) if "source_type" in data.columns else "No source_type column"

source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

In [55]:
data["confidence"].value_counts(dropna=False) if "confidence" in data.columns else "No confidence column"

confidence
high      40
medium     3
Name: count, dtype: int64

In [56]:
obs = data[data["record_type"]=="observation"].copy()
print("Observations:", obs.shape)

obs["indicator_code"].nunique(), sorted(obs["indicator_code"].dropna().unique())[:30]

Observations: (30, 34)


(19,
 ['ACC_4G_COV',
  'ACC_FAYDA',
  'ACC_MM_ACCOUNT',
  'ACC_MOBILE_PEN',
  'ACC_OWNERSHIP',
  'AFF_DATA_INCOME',
  'GEN_GAP_ACC',
  'GEN_GAP_MOBILE',
  'GEN_MM_SHARE',
  'USG_ACTIVE_RATE',
  'USG_ATM_COUNT',
  'USG_ATM_VALUE',
  'USG_CROSSOVER',
  'USG_MPESA_ACTIVE',
  'USG_MPESA_USERS',
  'USG_P2P_COUNT',
  'USG_P2P_VALUE',
  'USG_TELEBIRR_USERS',
  'USG_TELEBIRR_VALUE'])

In [57]:
# Coverage count per indicator
obs.groupby("indicator_code").size().sort_values()

indicator_code
ACC_MOBILE_PEN        1
AFF_DATA_INCOME       1
GEN_GAP_MOBILE        1
USG_CROSSOVER         1
USG_ATM_VALUE         1
USG_ATM_COUNT         1
USG_ACTIVE_RATE       1
GEN_MM_SHARE          1
USG_MPESA_USERS       1
USG_MPESA_ACTIVE      1
USG_TELEBIRR_USERS    1
USG_P2P_VALUE         1
USG_TELEBIRR_VALUE    1
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
GEN_GAP_ACC           2
USG_P2P_COUNT         2
ACC_FAYDA             3
ACC_OWNERSHIP         6
dtype: int64

In [58]:
# Date range (only if observation_date exists)
if "observation_date" in obs.columns:
    obs["observation_date"] = pd.to_datetime(obs["observation_date"], errors="coerce")
    obs["observation_date"].min(), obs["observation_date"].max()

In [59]:
events = data[data["record_type"]=="event"].copy()
print("Events:", events.shape)

# Show event fields (this is what you already prepared)
events.columns

Events: (10, 34)


Index(['record_id', 'record_type', 'category', 'pillar', 'indicator',
       'indicator_code', 'indicator_direction', 'value_numeric', 'value_text',
       'value_type', 'unit', 'observation_date', 'period_start', 'period_end',
       'fiscal_year', 'gender', 'location', 'region', 'source_name',
       'source_type', 'source_url', 'confidence', 'related_indicator',
       'relationship_type', 'impact_direction', 'impact_magnitude',
       'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country',
       'collected_by', 'collection_date', 'original_text', 'notes'],
      dtype='str')

In [60]:
# Quick event table
cols = [c for c in ["id","event_name","category","event_date","source_name","source_url","confidence"] if c in events.columns]
events[cols].sort_values(cols[-1] if "event_date" not in cols else "event_date").head(15)

,category,source_name,source_url,confidence
33,product_launch,Ethio Telecom,https://www.ethiotelecom.et/,high
34,market_entry,News,NaN,high
35,product_launch,Safaricom,NaN,high
36,infrastructure,NIDP,https://www.id.gov.et/,high
37,policy,NBE,NaN,high
38,milestone,EthSwitch,https://ethswitch.com/,high
39,partnership,EthSwitch,NaN,high
40,infrastructure,NBE/EthSwitch,NaN,high
41,policy,NBE,https://nbe.gov.et/,high
42,pricing,News,NaN,high


In [61]:
print("Impact links:", impact_links.shape)
impact_links.columns

Impact links: (14, 35)


Index(['record_id', 'parent_id', 'record_type', 'category', 'pillar',
       'indicator', 'indicator_code', 'indicator_direction', 'value_numeric',
       'value_text', 'value_type', 'unit', 'observation_date', 'period_start',
       'period_end', 'fiscal_year', 'gender', 'location', 'region',
       'source_name', 'source_type', 'source_url', 'confidence',
       'related_indicator', 'relationship_type', 'impact_direction',
       'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis',
       'comparable_country', 'collected_by', 'collection_date',
       'original_text', 'notes'],
      dtype='str')

In [62]:
cols = [c for c in ["parent_id","pillar","related_indicator","impact_direction","impact_magnitude","lag_months","evidence_basis"] if c in impact_links.columns]
impact_links[cols].head(15)

,parent_id,pillar,related_indicator,impact_direction,impact_magnitude,lag_months,evidence_basis
0,EVT_0001,ACCESS,ACC_OWNERSHIP,increase,high,12,literature
1,EVT_0001,USAGE,USG_TELEBIRR_USERS,increase,high,3,empirical
2,EVT_0001,USAGE,USG_P2P_COUNT,increase,high,6,empirical
3,EVT_0002,ACCESS,ACC_4G_COV,increase,medium,12,empirical
4,EVT_0002,AFFORDABILITY,AFF_DATA_INCOME,decrease,medium,12,literature
5,EVT_0003,USAGE,USG_MPESA_USERS,increase,high,3,empirical
6,EVT_0003,ACCESS,ACC_MM_ACCOUNT,increase,medium,6,theoretical
7,EVT_0004,ACCESS,ACC_OWNERSHIP,increase,medium,24,literature
8,EVT_0004,GENDER,GEN_GAP_ACC,decrease,medium,24,literature
9,EVT_0005,AFFORDABILITY,AFF_DATA_INCOME,increase,high,3,empirical


In [63]:
reference_codes.head()

,field,code,description,applies_to
0,record_type,observation,Actual measured value from a source,All
1,record_type,event,Policy launch market event or milestone,All
2,record_type,impact_link,Relationship between event and indicator (link...,All
3,record_type,target,Policy target or official goal,All
4,record_type,baseline,Starting point for comparison,All


In [64]:
# Try to infer how reference_codes is structured
reference_codes.columns

Index(['field', 'code', 'description', 'applies_to'], dtype='str')

In [65]:
if set(["field","value"]).issubset(reference_codes.columns):
    display(reference_codes["field"].value_counts())
    print("\nExample values for indicator_code:")
    display(reference_codes[reference_codes["field"]=="indicator_code"]["value"].head(30))
else:
    print("reference_codes format is different — we'll adapt after seeing columns.")

reference_codes format is different — we'll adapt after seeing columns.


In [66]:
# Task 1: Dataset overview

print("Record type counts:")
display(data["record_type"].value_counts())

print("\nPillar counts:")
display(data["pillar"].value_counts(dropna=False))

print("\nConfidence distribution:")
if "confidence" in data.columns:
    display(data["confidence"].value_counts(dropna=False))

Record type counts:


record_type
observation    30
event          10
target          3
Name: count, dtype: int64


Pillar counts:


pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64


Confidence distribution:


confidence
high      40
medium     3
Name: count, dtype: int64

In [67]:
# Task 1: Indicators coverage

obs = data[data["record_type"]=="observation"].copy()

print("Number of unique indicators:", obs["indicator_code"].nunique())

print("\nIndicator coverage (counts):")
display(obs.groupby("indicator_code").size().sort_values())

Number of unique indicators: 19

Indicator coverage (counts):


indicator_code
ACC_MOBILE_PEN        1
AFF_DATA_INCOME       1
GEN_GAP_MOBILE        1
USG_CROSSOVER         1
USG_ATM_VALUE         1
USG_ATM_COUNT         1
USG_ACTIVE_RATE       1
GEN_MM_SHARE          1
USG_MPESA_USERS       1
USG_MPESA_ACTIVE      1
USG_TELEBIRR_USERS    1
USG_P2P_VALUE         1
USG_TELEBIRR_VALUE    1
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
GEN_GAP_ACC           2
USG_P2P_COUNT         2
ACC_FAYDA             3
ACC_OWNERSHIP         6
dtype: int64

In [68]:
events = data[data["record_type"]=="event"].copy()
print("Event columns:", list(events.columns))

Event columns: ['record_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']


In [69]:
events = data[data["record_type"]=="event"].copy()
display(events[["record_id", "category", "indicator", "observation_date", "source_name", "confidence"]]
        .sort_values("observation_date"))

,record_id,category,indicator,observation_date,source_name,confidence
33,EVT_0001,product_launch,Telebirr Launch,2021-05-17,Ethio Telecom,high
41,EVT_0009,policy,NFIS-II Strategy Launch,2021-09-01,NBE,high
34,EVT_0002,market_entry,Safaricom Ethiopia Commercial Launch,2022-08-01,News,high
35,EVT_0003,product_launch,M-Pesa Ethiopia Launch,2023-08-01,Safaricom,high
36,EVT_0004,infrastructure,Fayda Digital ID Program Rollout,2024-01-01,NIDP,high
37,EVT_0005,policy,Foreign Exchange Liberalization,2024-07-29,NBE,high
38,EVT_0006,milestone,P2P Transaction Count Surpasses ATM,2024-10-01,EthSwitch,high
39,EVT_0007,partnership,M-Pesa EthSwitch Integration,2025-10-27,EthSwitch,high
42,EVT_0010,pricing,Safaricom Ethiopia Price Increase,2025-12-15,News,high
40,EVT_0008,infrastructure,EthioPay Instant Payment System Launch,2025-12-18,NBE/EthSwitch,high


In [70]:
from datetime import date
import pandas as pd

new_rows = []

# --- New Event (events use indicator + observation_date in your schema) ---
new_rows.append({
    "record_id": "EV_NEW_001",
    "record_type": "event",
    "category": "infrastructure",
    "pillar": None,

    "indicator": "Fayda Digital ID rollout expansion",
    "observation_date": "2024-01-01",

    "source_name": "Government of Ethiopia",
    "source_type": "website",
    "source_url": "https://id.gov.et",
    "confidence": "medium",

    "collected_by": "Hana Workneh",
    "collection_date": str(date.today()),
    "notes": "Digital ID reduces KYC friction and supports onboarding"
})

# --- New Observation 1 ---
new_rows.append({
    "record_id": "OB_NEW_001",
    "record_type": "observation",
    "pillar": "USAGE",
    "indicator": "Internet users (% of population)",
    "indicator_code": "ENB_INTERNET_USERS",
    "value_numeric": 25.0,
    "value_type": "numeric",
    "observation_date": "2023-01-01",
    "unit": "%",

    "source_name": "World Bank",
    "source_type": "dataset",
    "source_url": "https://data.worldbank.org",
    "confidence": "medium",

    "collected_by": "Hana Workneh",
    "collection_date": str(date.today()),
    "notes": "Proxy variable for digital payment readiness"
})

# --- New Observation 2 ---
new_rows.append({
    "record_id": "OB_NEW_002",
    "record_type": "observation",
    "pillar": "ACCESS",
    "indicator": "Electricity access (% of population)",
    "indicator_code": "ENB_ELECTRICITY_ACCESS",
    "value_numeric": 55.0,
    "value_type": "numeric",
    "observation_date": "2023-01-01",
    "unit": "%",

    "source_name": "World Bank",
    "source_type": "dataset",
    "source_url": "https://data.worldbank.org",
    "confidence": "low",

    "collected_by": "Hana Workneh",
    "collection_date": str(date.today()),
    "notes": "Enabler for digital finance adoption"
})

# Append to dataset
data_enriched = pd.concat([data, pd.DataFrame(new_rows)], ignore_index=True)

print("Dataset before:", data.shape)
print("Dataset after:", data_enriched.shape)

Dataset before: (43, 34)
Dataset after: (46, 34)


In [71]:
import os

os.makedirs("data/processed", exist_ok=True)

data_enriched.to_csv("data/processed/ethiopia_fi_unified_data_enriched.csv", index=False)

print("Saved enriched dataset to:")
print("data/processed/ethiopia_fi_unified_data_enriched.csv")

Saved enriched dataset to:
data/processed/ethiopia_fi_unified_data_enriched.csv
